In [75]:
import numpy as np
import pandas as pd

In [76]:
v_min = 2   # minimal volume
leaway_factor = 1.1    # factor for adjusting the volume
max_volume = 2000    # maximal solution volume

In [77]:
# raw_request_df = pd.DataFrame(np.array([[350, 1000], [250, 300], [100, 300]]))
raw_request_df = pd.read_csv("test.csv")

In [78]:
raw_request_df.columns.to_list() == ["concentration", "volume"]

True

In [79]:
raw_request_df

,concentration,volume
0,350,2000
1,300,200
2,250,200
3,225,200
4,200,200
5,180,200
6,160,200
7,140,200
8,120,200
9,100,200


In [80]:
request_df = raw_request_df.copy()
request_df["volume"][1:] = leaway_factor * raw_request_df["volume"][1:]

In [81]:
request_df

,concentration,volume
0,350,2000.0
1,300,220.0
2,250,220.0
3,225,220.0
4,200,220.0
5,180,220.0
6,160,220.0
7,140,220.0
8,120,220.0
9,100,220.0


In [107]:
pd.concat([request_df.iloc[:1], request_df.iloc[:1], request_df.iloc[1:]]).reset_index(drop=True)

,concentration,volume
0,350,2000.0
1,350,2000.0
2,300,220.0
3,250,220.0
4,225,220.0
5,200,220.0
6,180,220.0
7,160,220.0
8,140,220.0
9,120,220.0


In [82]:
idx_to_concentration = dict(zip(request_df.index, request_df["concentration"]))
idx_to_volume = dict(zip(request_df.index, request_df["volume"]))

Validity check

In [83]:
sum_series = request_df["concentration"] * request_df["volume"]

In [84]:
sum_series[0] > sum_series[1:].sum()

True

In [85]:
all(request_df["volume"] > 2 * v_min)

True

In [86]:
def find_solutions_two(min_concentrations, max_concentrations):
    boundary_to_count = {}
    current_count = 0
    for boundary in sorted(min_concentrations + max_concentrations):
        if boundary in min_concentrations:
            current_count += 1
        else:
            current_count -= 1
        boundary_to_count[boundary] = current_count
    min_count = len(min_concentrations) - max(boundary_to_count.values())
    
    a_left = 0
    a_right = 0
    
    for boundary in sorted(min_concentrations + max_concentrations):
        if boundary_to_count[boundary] >= min_count and a_left == 0:
            a_left = boundary
        if boundary_to_count[boundary] < min_count and a_left != 0:
            a_right = boundary

In [87]:
min_concentrations = []
max_concentrations = []
original_idxs = [0]

for j, v_j in idx_to_volume.items():
    if j == 0:
        continue
    min_concentrations.append((v_j / (v_j - v_min)) * idx_to_concentration[j])
    max_concentrations.append((v_j / v_min) * idx_to_concentration[j])
    
if idx_to_concentration[0] < max(min_concentrations):
    # too low
    raise Exception(f"original stock solution's concentration needs to be at least {max(min_concentrations):.2f}")
    
if max(min_concentrations) < min(max_concentrations):
    # one stock solution can cover all
    if idx_to_concentration[0] > min(max_concentrations):
        raise Exception(f"original stock solution needs to be diluted to {max(min_concentrations):.2f} to {min(max_concentrations):.2f}")
else:
    # need multiple stock solutions to dilute from
    # see if two can cover all
    # heuristic: find the median value between the current concentration and the smallest of min_concentrations
    new_stock_concentration = 0.5 * (min(min_concentrations), idx_to_concentration[0])
    for idx in range(1, len(min_concentrations)):
        if (new_stock_concentration > min_concentrations[idx] \
            and new_stock_concentration < max_concentrations[idx]) or \
            (idx_to_concentration[0] > min_concentrations[idx] \
            and idx_to_concentration[0] < max_concentrations[idx]):
            continue
        raise Exception(f"need more than 2 stock solutions with different concentrations")
    new_stock_line = pd.DataFrame({"concentration": new_stock_concentration, "volume": max_volume})
    request_df = pd.concat([request_df.iloc[:1], new_stock_line, request_df.iloc[1:]]).reset_index(drop=True)
    original_idxs.append(1)
    idx_to_concentration = dict(zip(request_df.index, request_df["concentration"]))
    idx_to_volume = dict(zip(request_df.index, request_df["volume"]))

Initialize

In [88]:
j_to_i = {}
v_need_dict = {idx: raw_request_df["volume"].iloc[idx] if idx != 0 else 0 \
               for idx in range(raw_request_df.shape[0])}
v_dilute_dict = {}
v_buffer_dict = {}

In [89]:
v_need_dict

{0: 0,
 1: 200,
 2: 200,
 3: 200,
 4: 200,
 5: 200,
 6: 200,
 7: 200,
 8: 200,
 9: 200,
 10: 200,
 11: 200,
 12: 200}

In [ ]:
def check_concentration(concentration, volume, v_min, ori_concentration):
    if (volume / (volume - v_min)) * concentration > ori_concentration:
        # too large
        return 1
    elif (volume / v_min) * concentration < ori_concentration:
        # j's concentration too small for the original stock solution, search for newer ones
        for i in range(1, j):
            if (idx_to_volume[j] / (idx_to_volume[j] - v_min)) * idx_to_concentration[j] \
                > idx_to_concentration[i]:
                # j's concentration too large for i, fail
                raise Exception(f"concentration at {j}-th row is too large ({idx_to_concentration[j]})")
            elif (idx_to_volume[j] / v_min) * idx_to_concentration[j] > idx_to_concentration[i]:
                j_to_i[j] = i
                break
        if i == j - 1:
            raise Exception(f"concentration at {j}-th row is too small ({idx_to_concentration[j]})")

In [90]:
for j in range(request_df.shape[0] - 1, 0, -1):
    if (idx_to_volume[j] / (idx_to_volume[j] - v_min)) * idx_to_concentration[j] > idx_to_concentration[0]:
        # j's concentration too large, fail
        raise Exception(f"concentration at {j}-th row is too large ({idx_to_concentration[j]})")
    elif (idx_to_volume[j] / v_min) * idx_to_concentration[j] < idx_to_concentration[0]:
        # j's concentration too small for the original stock solution, search for newer ones
        for i in range(1, j):
            if (idx_to_volume[j] / (idx_to_volume[j] - v_min)) * idx_to_concentration[j] \
                > idx_to_concentration[i]:
                # j's concentration too large for i, fail
                raise Exception(f"concentration at {j}-th row is too large ({idx_to_concentration[j]})")
            elif (idx_to_volume[j] / v_min) * idx_to_concentration[j] > idx_to_concentration[i]:
                j_to_i[j] = i
                break
        if i == j - 1:
            raise Exception(f"concentration at {j}-th row is too small ({idx_to_concentration[j]})")
    else:
        # best case scenario, can dilute from original stock solution
        j_to_i[j] = 0
    v_need_dict[j_to_i[j]] += (idx_to_concentration[j] / idx_to_concentration[j_to_i[j]]) * idx_to_volume[j]
    if v_need_dict[j_to_i[j]] > idx_to_volume[j_to_i[j]]:
        # the volume needed for i-th row is larger than the requested volume even after adjusting, fail
        raise Exception(f"volume needed for {j_to_i[j]}-th row is too large ({v_need_dict[j_to_i[j]]}), try requesting more")
    v_dilute_dict[j] = (idx_to_concentration[j] / idx_to_concentration[j_to_i[j]]) * idx_to_volume[j]
    v_buffer_dict[j] = ((idx_to_concentration[j_to_i[j]] - idx_to_concentration[j]) \
                        / idx_to_concentration[j_to_i[j]]) * idx_to_volume[j]

In [91]:
j_to_i

{12: 0, 11: 0, 10: 0, 9: 0, 8: 0, 7: 0, 6: 0, 5: 0, 4: 0, 3: 0, 2: 0, 1: 0}

In [92]:
idx_to_volume

{0: 2000.0,
 1: 220.00000000000003,
 2: 220.00000000000003,
 3: 220.00000000000003,
 4: 220.00000000000003,
 5: 220.00000000000003,
 6: 220.00000000000003,
 7: 220.00000000000003,
 8: 220.00000000000003,
 9: 220.00000000000003,
 10: 220.00000000000003,
 11: 220.00000000000003,
 12: 220.00000000000003}

In [93]:
v_buffer_dict

{12: 213.71428571428575,
 11: 204.2857142857143,
 10: 188.57142857142858,
 9: 157.14285714285717,
 8: 144.57142857142858,
 7: 132.0,
 6: 119.42857142857143,
 5: 106.85714285714288,
 4: 94.28571428571429,
 3: 78.57142857142858,
 2: 62.85714285714286,
 1: 31.42857142857143}

In [94]:
v_dilute_dict

{12: 6.2857142857142865,
 11: 15.714285714285715,
 10: 31.42857142857143,
 9: 62.85714285714286,
 8: 75.42857142857144,
 7: 88.00000000000001,
 6: 100.57142857142858,
 5: 113.14285714285715,
 4: 125.71428571428572,
 3: 141.42857142857144,
 2: 157.14285714285717,
 1: 188.57142857142858}

In [95]:
v_need_dict

{0: 1106.2857142857144,
 1: 200,
 2: 200,
 3: 200,
 4: 200,
 5: 200,
 6: 200,
 7: 200,
 8: 200,
 9: 200,
 10: 200,
 11: 200,
 12: 200}

Output

In [96]:
output_df = pd.concat([request_df, pd.DataFrame(columns=["dilution volume", "buffer volume", "from"])], sort=False)


In [97]:
for j, i in j_to_i.items():
    output_df.at[j, "from"] = i
    output_df.at[j, "buffer volume"] = v_buffer_dict[j]
    output_df.at[j, "dilution volume"] = v_dilute_dict[j]
output_df.at[0, "volume"] = v_need_dict[0]

In [98]:
output_df

,concentration,volume,dilution volume,buffer volume,from
0,350.0,1106.285714,NaN,NaN,NaN
1,300.0,220.000000,188.571,31.4286,0
2,250.0,220.000000,157.143,62.8571,0
3,225.0,220.000000,141.429,78.5714,0
4,200.0,220.000000,125.714,94.2857,0
5,180.0,220.000000,113.143,106.857,0
6,160.0,220.000000,100.571,119.429,0
7,140.0,220.000000,88,132,0
8,120.0,220.000000,75.4286,144.571,0
9,100.0,220.000000,62.8571,157.143,0
